In [3]:
from typing import List
import integer_funs
import torch
import evolutionary_operator as eo
import evolutionary_algorithm as ea
from evolutionary_algorithm import mutation_based_ea
from evolutionary_algorithm.student_experiment import StudentExperiment
import benchmark_funs as bf
dataset = list(map(bf.NegativeProxyObjective, bf.benchmark_functions)
)
dataset

[NegativeProxyObjective(
   (bf): Sphere()
 ),
 NegativeProxyObjective(
   (bf): Rosenbrock()
 ),
 NegativeProxyObjective(
   (bf): Step()
 ),
 NegativeProxyObjective(
   (bf): NoisyQuartic()
 ),
 NegativeProxyObjective(
   (bf): Schwefel226()
 ),
 NegativeProxyObjective(
   (bf): Ackley()
 ),
 NegativeProxyObjective(
   (bf): Schwefel222()
 ),
 NegativeProxyObjective(
   (bf): Holder()
 ),
 NegativeProxyObjective(
   (bf): Camel3()
 ),
 NegativeProxyObjective(
   (bf): Michal()
 )]

In [13]:
import torch
# mutation_ea = ea.mutation_based_ea.MutationBasedEA(problem)
# normal_choice = torch.Tensor([0, 1, -1, -1, 1]).to(torch.int64)
normal_choice = torch.Tensor([10, 3, 5, 5, 1]).to(torch.int64)
cauthy_choice = torch.Tensor([0, 2, -1, -1, 1]).to(torch.int64)

In [14]:
from tqdm import tqdm
def evaluate(dataset: List[bf.BenchmarkFunction], choice: torch.Tensor, experiment_times=30) -> torch.Tensor:
    result = torch.zeros(len(dataset), experiment_times)
    for i, b_fun in tqdm(enumerate(dataset), '使用数据集评估一个演化算法中'):
        mutation_ea = mutation_based_ea.MutationBasedEA(b_fun) 
        for j in range(experiment_times):
            result[i, j] = mutation_ea(choice)[1][-1]-b_fun.optival
    return result


In [15]:
normal_result = evaluate(dataset, normal_choice)
cauthy_result = evaluate(dataset, cauthy_choice)
normal_result, cauthy_result

演化算法优化中: 100%|██████████| 100/100 [00:00<00:00, 1718.13it/s]
使用数据集评估一个演化算法中: 10it [00:23,  2.37s/it]
演化算法优化中: 100%|██████████| 100/100 [00:00<00:00, 1855.80it/s]
使用数据集评估一个演化算法中: 10it [00:18,  1.86s/it]


(tensor([[-2.9395e+02, -5.1338e+02, -3.8053e+02, -4.8738e+02, -2.2231e+02,
          -5.6303e+02, -4.7713e+02, -4.5066e+02, -3.0084e+02, -7.4727e+02,
          -5.2239e+02, -3.6029e+02, -2.3111e+02, -2.8086e+02, -3.5128e+02,
          -4.6493e+02, -3.5231e+02, -4.2099e+02, -4.0563e+02, -4.0844e+02,
          -3.4561e+02, -2.6944e+02, -2.7744e+02, -9.9686e+02, -3.2079e+02,
          -3.2594e+02, -5.9121e+02, -1.9428e+02, -2.1898e+02, -2.4263e+02],
         [-1.2404e+04, -9.9097e+03, -4.1477e+03, -1.0660e+04, -9.2403e+04,
          -5.4188e+04, -1.8540e+05, -8.5559e+03, -2.0763e+04, -4.9315e+04,
          -8.3400e+04, -3.4834e+04, -1.2886e+04, -1.6787e+04, -8.6502e+04,
          -1.3900e+04, -4.4592e+04, -3.4734e+04, -3.7756e+04, -4.1178e+04,
          -4.3198e+04, -1.0228e+04, -1.7412e+05, -1.9624e+04, -3.6404e+04,
          -4.0917e+04, -3.3716e+04, -1.1199e+05, -4.3889e+04, -2.5090e+05],
         [-2.5900e+02, -7.1200e+02, -3.8200e+02, -2.3700e+02, -3.0100e+02,
          -4.5400e+02, 

In [16]:
normal_mean = torch.mean(normal_result, dim=1)
cauthy_mean = torch.mean(cauthy_result, dim=1)
from scipy.stats import wilcoxon
stat, p = wilcoxon(normal_result.T, cauthy_result.T, alternative='two-sided')
stat, p

/work/ssc-07/team/package_managers/anaconda3/envs/torch2.0/lib/python3.9/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


(array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 63.5,  0. ,  3. ]),
 array([1.86264515e-09, 1.86264515e-09, 1.86264515e-09, 1.86264515e-09,
        1.86264515e-09, 1.86264515e-09, 1.86264515e-09, 8.55558143e-04,
        1.86264515e-09, 5.23798844e-06]))

In [12]:
import pandas as pd
df = pd.DataFrame({'最终算法':normal_mean, 'cauthy':cauthy_mean, 'ranksum':stat, "p-value":p})
df.head()
df.to_excel("柯西与高斯对比.xlsx")